In [1]:
import os
import pandas as pd
import numpy as np
import re
import datetime
import pathlib

# custom package(s)
from utils import utils, synapse_login

import yaml
import pathlib

with open("./local_configs/notebook_config.yaml", "r") as f:
    config = yaml.safe_load(f)

csv_model = pathlib.Path("../" + config["file_names"]["csv_model"]).resolve()
json_model = pathlib.Path("../" + config["file_names"]["json_model"]).resolve()

In [68]:
dm_name = "../EL.data.model.csv"
output_dir = pathlib.Path("../backups").resolve()

dm_path = pathlib.Path(dm_name).resolve()
dm = utils.load_and_backup_dm(dm_path, output_dir)
# dm = dm.drop(columns=["Unnamed: 0"])

In [ ]:
dm = pd.read_csv('../backups/EL.data.model-2023-10-12.15-02.csv')

In [92]:
# clean up attributes
# remove any special characters
pattern = r"\(|\)|\.|:|;"
dm[['Attribute', 'DependsOn', 'Valid Values']] = dm[['Attribute', 'DependsOn', 'Valid Values']
                                                    ].apply(lambda x: x.str.replace(pattern, '').str.strip().str.replace('\s+', ''), axis=1)

In [94]:
# write out new model
dm.to_csv("../EL.data.model.csv", index=False)

In [95]:
# convert csv model to jsonld
!schematic schema convert {csv_model} --output_jsonld {json_model}

Starting schematic...
Done adding requirements and value ranges to attributes
The Data Model was created and saved to '/Users/nlee/Documents/Projects/ELITE/ELITE-data-models/EL.data.model.jsonld' location.
